<a href="https://colab.research.google.com/github/yhjune/Website-Fingerprinting/blob/main/load_pickle_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'



---


# mon_df1_contiuous.pkl : time, size

- sequence of packet sizes
- sequence of packet time stamps (packet size, direction)

In [2]:
import pandas as pd
import pickle

# Load the pickle file
with open("/content/drive/MyDrive/GitHub/global/dataset/mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    mon_data = pickle.load(fi)

print("Load mon_standard")

Load mon_standard


In [ ]:
#print(df.keys()) # keys :
a=list(mon_data.items())
print(len(a)) # 95 websites * 10 subpages = 950
print(len(a[0]),a[0] ) # 2 # 0, [[1,2,3,..],[1,2,3..],...,[1,2,3...]] |  0~949 | 0~9 eq website1 | 10~19 eq website2 |
print(len(a[0][1]),a[0][1]) # 20 | observed 20 times each
print(len(a[0][1][19]),a[0][1][19]) # timestamps with direction, not equals between each observations

In [7]:
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

mon_time_seq = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
mon_size_seq = [] # Array to store instances (direction*size) - size information
mon_label = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label.
        # Thus, URLs fetched from the same site are labeled identically.
    for sample in mon_data[i]:
        size_seq = []
        time_seq = []
        sum = 0
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
            sum += dr * 512

        mon_time_seq.append(time_seq)
        mon_size_seq.append(size_seq)
        mon_label.append(label)

df1 = pd.DataFrame(zip(mon_time_seq,mon_size_seq,mon_label),columns=['time_seq','size_seq','label'])

In [8]:
df1

,time_seq,size_seq,label
0,"[0.0, 0.14, 0.14, 0.31, 0.31, 0.51, 0.51, 0.51...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
1,"[0.0, 0.13, 0.13, 0.31, 0.77, 1.11, 1.11, 1.11...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
2,"[0.0, 0.11, 0.11, 0.23, 0.97, 1.11, 1.11, 1.11...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
3,"[0.0, 0.27, 0.27, 0.6, 0.6, 0.88, 0.89, 0.89, ...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
4,"[0.0, 0.11, 0.11, 0.36, 0.36, 0.6, 0.6, 0.6, 0...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
...,...,...,...
18995,"[0.0, 0.15, 0.15, 0.33, 0.91, 1.12, 1.13, 1.13...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",94
18996,"[0.0, 0.16, 0.16, 0.35, 0.99, 1.26, 1.26, 1.26...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",94
18997,"[0.0, 0.11, 0.11, 0.36, 0.36, 0.83, 0.83, 0.83...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",94
18998,"[0.0, 0.17, 0.17, 0.32, 1.98, 2.56, 2.56, 2.56...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",94


## Export Pickle

In [9]:
cd /content/drive/MyDrive/GitHub/global

/content/drive/MyDrive/GitHub/global


In [10]:
picklefile = open('mon_df1_contiuous.pkl','wb')
pickle.dump(df1,picklefile)
picklefile.close()

# mon_df2_contiuous.pkl : cumluative size, burst

- sequence of cumluative packet sizes
- sequence of bursts
- label


In [1]:
import pandas as pd
import pickle

# Load the pickle file
with open("/content/drive/MyDrive/GitHub/global/dataset/mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    mon_data = pickle.load(fi)
print("Load mon_standard")

Load mon_standard


In [2]:
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

mon_label = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]
mon_cumul = []
mon_burst = []

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label.
        # Thus, URLs fetched from the same site are labeled identically.
    for sample in mon_data[i]:
        size_seq = []
        cumul_seq = []
        burst_seq = []
        sum = 0
        for c in sample:
            dr = 1 if c > 0 else -1
            size_seq.append(dr * 512)
            sum += dr * 512
            cumul_seq.append(sum)

        mon_cumul.append(cumul_seq)
        mon_label.append(label)

        burst = 1
        for j in range(len(size_seq)-1):
            if size_seq[j] == size_seq[j+1]:
              burst += 1
            elif j == len(size_seq)-1:
              burst_seq.append(size_seq[j] * burst)
              burst = 1
            else:
              burst_seq.append(size_seq[j] * burst)
              burst = 1

        mon_burst.append(burst_seq)

df2 = pd.DataFrame(zip(mon_label,mon_cumul,mon_burst),columns=['label','cumlative_seq','burst_seq'])

In [3]:
df2

,label,cumlative_seq,burst_seq
0,0,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7168, 512..."
1,0,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7168, 512..."
2,0,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7168, 512..."
3,0,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7168, 512..."
4,0,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7168, 512..."
...,...,...,...
18995,94,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7680, 102..."
18996,94,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7680, 102..."
18997,94,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7680, 153..."
18998,94,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 1024, -7680, 102..."


## Export Pickle

In [4]:
cd /content/drive/MyDrive/GitHub/global

/content/drive/MyDrive/GitHub/global


In [5]:
picklefile = open('mon_df2_contiuous.pkl','wb')
pickle.dump(df2,picklefile)
picklefile.close()

# mon_df3_categorical : packet nums, income, outcome
columns :
- number of packet
- total incoming
- total outgoing

incoming : server to client

outcoming : client to server

In [6]:
import pickle

with open("/content/drive/MyDrive/GitHub/global/mon_df1_contiuous.pkl", 'rb') as fi:
    df1 = pickle.load(fi)
print("Load mon_df1_contiuous")

Load mon_df1_contiuous


In [7]:
# number of packet, total incoming, total outgoing
import pandas as pd

packet_num = []
total_income = []
total_outcome = []

for i in range(len(df1['size_seq'])):
  num = len(df1['size_seq'][i])
  income = sum(x > 0 for x in list(df1['size_seq'][i]))
  outcome = sum(x < 0 for x in list(df1['size_seq'][i]))
  packet_num.append(num)
  total_income.append(income)
  total_outcome.append(outcome)

df3 = pd.DataFrame(zip(packet_num,total_income,total_outcome), columns =['packet_num','total_income','total_outcome'])
print(f'total : {len(packet_num)}')

total : 19000


## Export Pickle

In [8]:
cd /content/drive/MyDrive/GitHub/global

/content/drive/MyDrive/GitHub/global


In [9]:
picklefile = open('mon_df3_categorical.pkl','wb')
pickle.dump(df3,picklefile)
picklefile.close()

# mon_df4_custom.pkl

`df4 = pd.DataFrame({'column name1': df1['df1_col'], 'column name2': df2['Name'], ... })`

In [ ]:
import pickle

with open("/content/drive/MyDrive/GitHub/global/mon_df1_contiuous.pkl.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    df1 = pickle.load(fi)
print("Loading mon_df1_contiuous...")


with open("/content/drive/MyDrive/GitHub/global/mon_df2_contiuous.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    df2 = pickle.load(fi)
print("Loading mon_df2_contiuous...")

with open("/content/drive/MyDrive/GitHub/global/mon_df3_categorical.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    df3 = pickle.load(fi)
print("Loading mon_df3_categorical...")


In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()

In [ ]:
import pandas as pd
df4 = pd.DataFrame({'label': df1['label'], 'packet_num': df3['packet_num']})
df4.head()

## Export Pickle

In [ ]:
cd /content/drive/MyDrive/GitHub/global

In [ ]:
picklefile = open('mon_df4_custom.pkl','wb')
pickle.dump(df4,picklefile)
picklefile.close()